# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep
from pprint import pprint

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlists

In [2]:
playlist = {
    ## Study ##
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    ## Probation ##
    'zi_Over' : "3ackEOD2vox5Oc1vjeALKJ",
    # 'zzzDupe' : "1VPXM7m1by79EdEzDqGsHy",
    # 'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

dupeDump = "1VPXM7m1by79EdEzDqGsHy"
dupeDict = { 'DupeDump' : dupeDump }
backfill = playlist['zi_Over']


# Playlist Functions

In [3]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']
    

def fetch_entire_playlist( playlist_ID ):
    """ Get infodump on all plalist tracks """
    plTracks = []
    trCount  = 0
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    Ntracks = response['total']
    while 1:
        trCount += len(response['items'])
        plTracks.extend( response['items'] )
        
        if trCount >= Ntracks:
            break
    
        response = spot.user_playlist_tracks(
            CLIENT_ID, 
            playlist_ID, 
            fields = 'items,uri,name,id,total', 
            limit  = _RESPONSE_LIMIT,
            offset = trCount
        )
    return plTracks


def create_dupe_removal_jobs( plDict, pause_s = 0.5 ):
    """ Scan each playlist for duplicates within itself and from previous in the sorted playlists, Return removal task list """    
    uniqList = list()
    srtdKeys = sorted( list( plDict.keys() ) )
    dumpList = list()
    plIDlist = list()

    def p_exists_in_prev( itemID, currDex ):
        """ Return True if `itemID` exists in a playlist before `currDex`, Otherwise return False """
        for plSet in uniqList[ :currDex ]:
            if itemID in plSet:
                return True
        return False

    # 1. For every playlist in the dict, do
    for i, plName in enumerate( srtdKeys ):

        print( f"##### Playlist {i+1}: {plName} #####" )
        
        # 2. Fetch playlist and establish a running set
        plID      = plDict[ plName ]
        trkList_i = fetch_entire_playlist( plID )
        trkSet_i  = set([])
        dumpLst_i = list()
        plIDlist.append( plID )
        origLen_i = len( trkList_i )
        print( f"Fetched {origLen_i} tracks!" )
        
        # 3. For every track j in playlist i, do
        j = 0
        while j < len( trkList_i ):

            track_j  = trkList_i[j]
            p_dump_j = False
            
            # 4. Test 1: Did we find this song earlier in the playlist?
            trackID_j = track_j['track']['id']
            if trackID_j in trkSet_i:
                dumpLst_i.append( (trackID_j, j,) )
                p_dump_j = True
                trkList_i.pop(j)

            # 5. Test 2: Did we find this song in and earlier playlist?
            if (not p_dump_j) and p_exists_in_prev( trackID_j, i ):
                dumpLst_i.append( (trackID_j, j,) )
                p_dump_j = True
                trkList_i.pop(j)
            
            # 6. Uniqify
            if not p_dump_j:
                trkSet_i.add( trackID_j )
                j += 1
            
        # 7. Store track set i and dump list
        uniqList.append( trkSet_i )
        dumpList.append( dumpLst_i )
        print( f"In {plName}/{plID}: Retain {len(trkSet_i)}, Dump {len(dumpLst_i)}, Valid? {(len(trkSet_i)+len(dumpLst_i))==origLen_i}\n" )
        sleep( pause_s )

    # N. Return removal job list
    return list( zip( plIDlist, dumpList ) )
            

def run_dupe_removal_jobs( jobList, plTarget = None, pause_s = 0.5 ):
    """ Run jobs created in `create_dupe_removal_jobs` and optionally store them in `plTarget` in the event of a massive fuckup """
    print( f"########## About to run {len(jobList)} jobs ... ##########\n" )
    for i, (plID_i, remLst_i) in enumerate( jobList ):
        print( f"##### Job {i+1}: {plID_i}, {len(remLst_i)} to remove ... #####" )

        if len( remLst_i ):
            remvLs_i = [item[0] for item in remLst_i]

            if plTarget is not None:
                remvLs_ii = remvLs_i[:]
                while len( remvLs_ii ):
                    if len( remvLs_ii ) > _RESPONSE_LIMIT:
                        remvLs_jj = remvLs_ii[:_RESPONSE_LIMIT]
                        remvLs_ii = remvLs_ii[_RESPONSE_LIMIT:]
                    else:
                        remvLs_jj = remvLs_ii[:]
                        remvLs_ii = list()
                    
                    print( f"Venting dupes to {plTarget} ..." )
                    spot.user_playlist_add_tracks( CLIENT_ID, plTarget, remvLs_jj )
                    sleep( pause_s )
    
            print( f"removing dupes from {plID_i} ..." )
            for (trackID, j) in remLst_i:
                
                res = spot.playlist_remove_specific_occurrences_of_items( 
                    plID_i, 
                    [{'uri': trackID, 'positions':[j,]},]
                )
                print( "\tRemove:", trackID, j, res )
                sleep( pause_s )
        print()
    print( f"########## Completed {len(jobList)} jobs! ##########\n" )


def backfill_all_to_limit( plDict, plBckFll, N_limit = 400, pause_s = 0.5 ):
    """ Use `plBckFll` to top off each playlist in `plDict` up to `N_limit` """
    
    srtdKeys = sorted( list( plDict.keys() ) )
    fillBank = fetch_entire_playlist( plBckFll )
    NbkFlBgn = len( fillBank )
    Nfill    = 0

    print( f"\n########## Beginning with a backfill bank of {NbkFlBgn} items! ##########\n" )
    
    # 1. For every playlist in the dict, do
    for i, plName in enumerate( srtdKeys ):
        print( f"##### Playlist {i+1}: {plName} #####" )
        
        # 2. Calc backfill need
        plID = plDict[ plName ]
        plN  = get_playlist_length( plID )
        
        # 3. If backfill needed, perform backfill
        if (plN < N_limit) and (plID != plBckFll):
            Nbf   = N_limit - plN
            if Nbf > len( fillBank ):
                print( f"\tWARN: Needed {Nbf} tracks, but only {len( fillBank )} available! Skipping playlist ...\n" )
                continue
            Nfill += Nbf
            bkLs  = [item['track']['id'] for item in fillBank[ :Nbf ]]

            # 4. Top off playlist
            res = spot.user_playlist_add_tracks( CLIENT_ID, plID, bkLs )
            print( f"\tAdd {len(bkLs)} tracks: {res}" )
            sleep( pause_s )

            # 4. Pop from backfill bank
            for bkfl_j in bkLs:
                res = spot.playlist_remove_specific_occurrences_of_items( 
                    plBckFll, 
                    [{'uri': bkfl_j, 'positions':[0,]},]
                )
                print( "\tRemove:", bkfl_j, res )
                sleep( pause_s )

            fillBank = fillBank[ Nbf: ]
        print()

    NbkFlEnd = get_playlist_length( plBckFll )
    print( f"########## Ending with a backfill bank of {NbkFlEnd} items!, Valid?: {(NbkFlBgn)==(NbkFlEnd+Nfill)} ##########\n" )
                
    
        

# Identify Duplicates

In [4]:
jobList = create_dupe_removal_jobs( playlist )

##### Playlist 1: study01 #####
Fetched 400 tracks!
In study01/0a2qoe6S7lYeZ6nlhZdA0v: Retain 400, Dump 0, Valid? True

##### Playlist 2: study02 #####
Fetched 400 tracks!
In study02/6gbtR2cBq5PvkghidCvvGk: Retain 400, Dump 0, Valid? True

##### Playlist 3: study03 #####
Fetched 400 tracks!
In study03/3o3lN2qntdEV7UKTuuC77K: Retain 396, Dump 4, Valid? True

##### Playlist 4: study04 #####
Fetched 422 tracks!
In study04/41sFSisljvBDMBXtpp5NIw: Retain 419, Dump 3, Valid? True

##### Playlist 5: study05 #####
Fetched 394 tracks!
In study05/02iS5AFGp8YVuUUqcQf8ys: Retain 394, Dump 0, Valid? True

##### Playlist 6: study06 #####
Fetched 400 tracks!
In study06/6KI7A4MWrSM7EyKRUjxIi1: Retain 400, Dump 0, Valid? True

##### Playlist 7: study07 #####
Fetched 400 tracks!
In study07/3V055Md2JdrUT8tX0af7di: Retain 398, Dump 2, Valid? True

##### Playlist 8: study08 #####
Fetched 399 tracks!
In study08/0tspdJlwSgiyf2O9PO6QaP: Retain 398, Dump 1, Valid? True

##### Playlist 9: study09 #####
Fetched 

# Move Duplicates

In [5]:
run_dupe_removal_jobs( jobList, dupeDump, pause_s = 0.5 )

########## About to run 19 jobs ... ##########

##### Job 1: 0a2qoe6S7lYeZ6nlhZdA0v, 0 to remove ... #####

##### Job 2: 6gbtR2cBq5PvkghidCvvGk, 0 to remove ... #####

##### Job 3: 3o3lN2qntdEV7UKTuuC77K, 4 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 3o3lN2qntdEV7UKTuuC77K ...
	Remove: 6MErUIO6iex1QFO6gqr5Ai 370 {'snapshot_id': 'AAADLrgkgP3PrYsyg+fMfRxdTQYXhtsn'}
	Remove: 5T4ESVKmwpomAgyUuEtX3t 381 {'snapshot_id': 'AAADLxzdEAexci3mqf7VX+BCDYxY0tyt'}
	Remove: 33zwRLZHgDIgijylwRro02 382 {'snapshot_id': 'AAADMH51rL2IXZtxlcLpeyV9gPI0e9Ys'}
	Remove: 6NLd7wuYEBrTEJkqPbjv4C 395 {'snapshot_id': 'AAADMQ9+KpUP/AgzF3XjOLwvkvFJdIWy'}

##### Job 4: 41sFSisljvBDMBXtpp5NIw, 3 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 41sFSisljvBDMBXtpp5NIw ...
	Remove: 4Z4r2CuS71goyw6rAdtK3Y 125 {'snapshot_id': 'AAADm1i2jhmKL1JrNY3/FSsKX0U25wKT'}
	Remove: 7JB4HInStR5X4K7y5gE8fd 365 {'snapshot_id': 'AAADnHJmI2KakegVcR1Y6inO6qbifIlA'}
	Re

# Backfill

In [6]:
backfill_all_to_limit( playlist, backfill, N_limit = 400, pause_s = 0.5 )


########## Beginning with a backfill bank of 275 items! ##########

##### Playlist 1: study01 #####

##### Playlist 2: study02 #####

##### Playlist 3: study03 #####
	Add 4 tracks: {'snapshot_id': 'AAADMpcG89KXVXX0xvHvGm3DCI32dC5C'}
	Remove: 4OnuNLwvIgroDdTXX2k5dF {'snapshot_id': 'AAAA/Vky4wmYOnapqRqVl+l7ImDfEJBC'}
	Remove: 1bweOqaO6SIGRRMupc7zMm {'snapshot_id': 'AAAA/rth0F5tObpUbWZs7cWA06dnA61Z'}
	Remove: 2VTW7gB4G7pm6zvlXjzQnt {'snapshot_id': 'AAAA/1cTzadOscYD2A2J4JhVJr1h1enu'}
	Remove: 6AQzb8kGufof2uRJbJD8my {'snapshot_id': 'AAABALer1cdx+Y8IJpiJ2yxKVF0xsfX+'}

##### Playlist 4: study04 #####

##### Playlist 5: study05 #####
	Add 6 tracks: {'snapshot_id': 'AAAFv8SijfilvzwwAlGVUI61TMmpq2lW'}
	Remove: 26gYGootyi3t9h6xfbnyVT {'snapshot_id': 'AAABAR1Vsud0ib3JTw6K9GpI+GmcB3jy'}
	Remove: 2sGRAsAhd08FXge9fxSknO {'snapshot_id': 'AAABAkYn2x4by8+KHfZoB2utxcssbTR9'}
	Remove: 6TbzfeKHyCKNjdBCXps2aR {'snapshot_id': 'AAABAyNIVe6XlQ3VYXoJJzmJi/6AaiO/'}
	Remove: 1ggNWdr63cOMycbtsFamV3 {'snapshot_id

# Clean Dupes

In [7]:
dupeCleanJob = create_dupe_removal_jobs( dupeDict )

##### Playlist 1: DupeDump #####
Fetched 662 tracks!
In DupeDump/1VPXM7m1by79EdEzDqGsHy: Retain 609, Dump 53, Valid? True



In [8]:
run_dupe_removal_jobs( dupeCleanJob, None, pause_s = 0.5 )

########## About to run 1 jobs ... ##########

##### Job 1: 1VPXM7m1by79EdEzDqGsHy, 53 to remove ... #####
removing dupes from 1VPXM7m1by79EdEzDqGsHy ...
	Remove: 6MErUIO6iex1QFO6gqr5Ai 491 {'snapshot_id': 'AAACPBZICwG0sGcUyxIapqkgPZ3ItMJF'}
	Remove: 5T4ESVKmwpomAgyUuEtX3t 491 {'snapshot_id': 'AAACPVmc96MAlfR7s1BTtKdnMsddzRPu'}
	Remove: 6NLd7wuYEBrTEJkqPbjv4C 492 {'snapshot_id': 'AAACPvpHJ8UoBmyGt7cdjJfx7tv9gwi+'}
	Remove: 5T4ESVKmwpomAgyUuEtX3t 500 {'snapshot_id': 'AAACPvpHJ8UoBmyGt7cdjJfx7tv9gwi+'}
	Remove: 1iuLBLfB0jfKNcsorxMxWm 502 {'snapshot_id': 'AAACP3DCC+FO3wDgcQz6JububwvwX9sq'}
	Remove: 28jwp7ime5xnL3wjMqktKT 513 {'snapshot_id': 'AAACQHxQOLu/SvgvcRPXgM4ArSkdy3TX'}
	Remove: 7kCPTBNgbcgbqiwzX12FGy 515 {'snapshot_id': 'AAACQTIz/ezVBhQJiKFGTXrDwFQ6ixoM'}
	Remove: 1iQAU5rssr0ogNzPbL8af9 515 {'snapshot_id': 'AAACQtaq6a5uRbRZPkfAhSRuX5oxClKw'}
	Remove: 2l6mwD8aBjhsRHq67BsLHq 518 {'snapshot_id': 'AAACQ+Ga781g73G9VWz0n6CTIPYyodR3'}
	Remove: 6idiBkxfhZJqADlg3ExDcL 520 {'snapshot_id': 'A